In [1]:
import os
import cv2
from codecs import open as copen
from shutil import copyfile, rmtree

from sklearn.model_selection import train_test_split
from django.db.models import Q
from django.contrib.auth.models import User

from app.models import Label,Image,Batch, Comment, STATUS_CHOICES
from app.models import TODO, TAGGING, REVIEWING, DONE

In [2]:
dataset_path = os.path.join('.', 'static', 'dataset')
yolo_path = os.path.join('.', 'yolo')
yolo_anno_dir = os.path.join(yolo_path, 'annotate')
obj_path = os.path.join(yolo_anno_dir, 'obj')

if os.path.exists(obj_path):
    rmtree(obj_path)
os.makedirs(obj_path)

In [3]:
classcnt = 0
classidx = dict()
stop = ['/.', '.', './', '\.', '\.']

def get_class(label):
    global classidx, classcnt
    info = [label.brand, label.model]
#     print(info)
#     cls = "{}.{}".format(*info)
    cls = label.brand
    print
    if not any(info) or cls in stop:
        cls = "unknown"
    if not cls in classidx:
        classidx[cls] = classcnt
        classcnt += 1
    return cls

def get_coord(label):
    return label.x, label.y, label.width, label.height

def get_yolo_anno(mat, label):
    imh, imw = mat.shape[:2]
    x, y, w, h = get_coord(label)
    print(x, y, w, h)
    cls = get_class(label)
    dw = 1.0/imw
    dh = 1.0/imh
    x = (x+(x+w))/2.0
    y = (y+(y+h))/2.0
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    
#     x, y, w, h = get_coord(label)
#     rx = (x+(w/2.0))/imw
#     ry = (y+(h/2.0))/imh
#     rw = w/2.0/imw
#     rh = h/2.0/imh
#     cls = get_class(label)
    return classidx[cls], x, y, w, h

def recheck_ratio(mat, labels, filename='test.png'):
    imh, imw = mat.shape[:2]
    print(imh)
    tmp = mat.copy()
    for label in labels:
        cls, rx, ry, rw, rh = label
        w = int(rw*imw)
        h = int(rh*imh)
        x1 = int(rx*imw-w/2.0)
        y1 = int(ry*imh-h/2.0)
        x2 = int(x1+w)
        y2 = int(y1+h)
        tmp = cv2.rectangle(tmp, (x1, y1), (x2, y2), (0, 255, 0))
        w = int(rw*imw)
        h = int(rh*imh)
        x = int(rx*imw)
        y = int(ry*imh)
        tmp = cv2.circle(tmp, (x, y), h, (0,0,255))
    cv2.imwrite(filename, tmp)


In [4]:
images = Image.objects.filter(batch__status=REVIEWING)\
    .exclude(label__isnull=True)
    
for i, image in enumerate(images):
    labels = list()
    labelled =  Label.objects.filter(image=image)
    image_path = os.path.join(dataset_path, image.src_path)
    fname, ext = os.path.splitext(image.src_path)
    fname = image.id
    obj_img_path = os.path.join(obj_path, "{}.png".format(fname))
    obj_txt_path = os.path.join(obj_path, "{}.txt".format(fname))
    mat = cv2.imread(image_path)
    for label in labelled:
        ret = get_yolo_anno(mat, label)
        labels.append(ret)
        
#     !!!DEBUG
    recheck_ratio(mat, labels, "./yolo/{}.png".format(i))
    copyfile(image_path, obj_img_path)
    with open(obj_txt_path, 'w+') as fp:
        for label in labels:
            fp.write(" ".join([str(x) for x in label]))
            fp.write(os.linesep)




(5, 318, 1300, 759)

1080
(939, 203, 586, 539)

1080
(1063, 227, 759, 651)

(1146, 537, 152, 111)

(0, 7, 1089, 1062)

1080
(1105, 261, 563, 464)

(1218, 435, 112, 67)

1080
(1005, 14, 732, 760)

(1251, 594, 98, 63)

(1651, 213, 216, 265)

1080
(763, 207, 403, 662)

(775, 607, 81, 82)

1080
(1303, 181, 429, 384)

(1437, 434, 87, 47)

1080
(797, 219, 928, 682)

(963, 533, 146, 105)

1080
(1257, 241, 472, 416)

(1373, 402, 89, 55)

1080
(1558, 325, 239, 207)

1080
(1427, 277, 337, 307)

(1522, 399, 64, 39)

(1, 602, 771, 474)

1080
(649, 285, 456, 633)

(662, 710, 80, 95)

(1547, 325, 219, 211)

(1361, 309, 192, 159)

(1611, 418, 67, 40)

(1398, 369, 47, 28)

(899, 238, 489, 306)

1080
(1247, 322, 246, 443)

(1286, 597, 68, 67)

1080
(0, 0, 1615, 1071)

1080
(1086, 294, 599, 464)

(1209, 487, 128, 71)

1080
(1123, 278, 622, 494)

(1285, 530, 115, 67)

(826, 245, 490, 307)

(919, 442, 57, 40)

1080
(369, 219, 325, 250)

(501, 393, 76, 40)

(243, 201, 175, 171)

(313, 310, 64, 27)

(610, 1

(1149, 342, 240, 415)

(819, 239, 555, 278)

(919, 441, 54, 39)

(1351, 302, 214, 163)

(1747, 278, 107, 140)

1080
(1, 357, 1424, 717)

(1325, 606, 585, 464)

(1414, 297, 187, 157)

1080
(414, 181, 292, 501)

1080
(278, 206, 603, 519)

(583, 519, 133, 69)

1080
(729, 201, 240, 515)

1080
(1117, 270, 636, 454)

(1278, 563, 116, 81)

(1, 687, 463, 390)

1080
(113, 325, 1524, 751)

(217, 722, 113, 138)

(1365, 305, 203, 145)

1080
(915, 258, 751, 582)

(1027, 554, 159, 104)

1080
(3, 246, 1226, 824)

1080
(1087, 141, 673, 548)

(1266, 349, 124, 73)

1080
(1173, 302, 540, 412)

(1289, 565, 105, 75)

(811, 249, 501, 284)

1080
(1250, 110, 455, 520)

(1683, 265, 107, 145)

1080
(475, 337, 1131, 684)

(625, 631, 113, 102)

(1407, 305, 178, 147)

(1451, 366, 33, 27)

1080
(1299, 309, 457, 357)

(1386, 541, 100, 60)

(835, 262, 539, 284)

(169, 483, 372, 255)

1080
(182, 202, 442, 396)

(334, 463, 127, 69)

1080
(714, 281, 923, 685)

(878, 74, 379, 998)

(82, 251, 368, 818)

(1610, 331, 183, 1

(967, 25, 777, 757)

(286, 455, 222, 283)

1080
(851, 5, 927, 784)

1080
(161, 329, 1189, 736)

(261, 666, 113, 138)

1080
(1226, 310, 518, 402)

(1353, 574, 96, 55)

(1751, 314, 53, 75)

1080
(761, 2, 1028, 882)

1080
(599, 270, 1045, 718)

(765, 570, 136, 96)

1080
(861, 513, 372, 560)

(961, 279, 295, 523)

(362, 265, 182, 593)

(1530, 329, 270, 228)

(1745, 302, 112, 143)

(1403, 281, 161, 163)

(1454, 355, 39, 26)

(1599, 419, 67, 45)

(1062, 918, 146, 132)

1080
(110, 221, 383, 316)

(262, 413, 91, 44)

(642, 170, 244, 202)

(751, 302, 43, 27)

(442, 199, 100, 150)

1080
(3, 3, 505, 1071)

(1625, 251, 263, 273)

1080
(1442, 282, 296, 786)

(1410, 299, 143, 135)

(1455, 353, 41, 33)

1080
(1199, 21, 546, 617)

1080
(1286, 327, 438, 314)

(1395, 446, 89, 51)

1080
(1, 6, 1155, 1071)

1080
(3, 314, 1551, 763)

(1478, 330, 282, 220)

(1566, 406, 56, 44)

1080
(1311, 238, 477, 391)

(1549, 142, 157, 138)

(1443, 418, 87, 56)

(3, 481, 1305, 592)

1080
(565, 119, 1077, 798)

(726, 470,

(1090, 0, 712, 707)

(831, 283, 307, 270)

(917, 446, 61, 32)

1080
(157, 167, 331, 320)

(259, 346, 82, 46)

(526, 206, 79, 146)

1080
(0, 1, 1646, 1076)

1080
(1346, 234, 443, 316)

(1534, 146, 173, 98)

1080
(1082, 257, 617, 440)

(1225, 450, 114, 64)

1080
(201, 238, 448, 364)

(379, 487, 121, 62)

1080
(1087, 313, 666, 476)

(1199, 635, 121, 83)

1080
(1366, 275, 366, 257)

(1467, 374, 70, 50)

1080
(699, 511, 373, 285)

(1338, 261, 428, 344)

(1457, 473, 77, 49)

(906, 273, 467, 279)

1080
(206, 218, 778, 631)

(607, 643, 173, 93)

1080
(1202, 275, 519, 367)

(1397, 500, 78, 43)

1080
(200, 177, 1717, 899)

1080
(1492, 126, 329, 431)

1080
(1487, 289, 301, 243)

(288, 441, 219, 286)

1080
(1045, 338, 666, 441)

(1203, 514, 110, 71)

1080
(578, 477, 476, 360)

(587, 623, 86, 83)

(1443, 418, 161, 190)

(1654, 214, 192, 251)

(1406, 311, 98, 119)

1080
(1330, 169, 419, 385)

(1463, 321, 77, 49)

1080
(486, 201, 1216, 768)

(714, 630, 140, 104)

(1671, 231, 162, 242)

(1683, 374, 61

(43, 297, 1539, 779)

(106, 699, 118, 153)

(1669, 334, 149, 124)

(1406, 319, 182, 130)

(1713, 393, 40, 24)

(1450, 366, 35, 20)

1080
(1370, 318, 406, 312)

(1481, 529, 79, 55)

1080
(291, 195, 770, 647)

(715, 633, 179, 95)

1080
(829, 426, 348, 326)

(877, 538, 91, 80)

(950, 238, 202, 291)

1080
(1, 5, 1667, 1069)

(1633, 317, 161, 196)

1080
(610, 326, 1043, 680)

(1678, 338, 150, 110)

(1722, 419, 36, 22)

(738, 614, 143, 124)

(609, 338, 106, 155)

(1515, 329, 133, 89)

1080
(474, 509, 113, 54)

(211, 28, 533, 586)

1080
(848, 266, 523, 283)

(923, 449, 52, 35)

(1614, 348, 200, 139)

(1362, 327, 202, 129)

(677, 448, 145, 184)

(1669, 408, 47, 24)

(1404, 373, 34, 23)

(767, 494, 45, 41)

1080
(826, 224, 835, 603)

(283, 436, 227, 292)

(151, 287, 173, 198)

(970, 472, 139, 80)

1080
(178, 200, 308, 282)

(288, 409, 81, 37)

1080
(111, 7, 601, 586)

1080
(150, 11, 456, 506)

(313, 371, 97, 55)

1080
(297, 221, 1065, 808)

(997, 794, 176, 107)

1080
(199, 217, 558, 451)

(465,

...

(1416, 323, 342, 259)

(1521, 464, 56, 31)

1080
(0, 1, 1667, 1079)

1080
(1616, 195, 227, 300)

1080
(81, 156, 1581, 923)

(184, 811, 149, 152)

1080
(1230, 477, 353, 204)

(1250, 597, 66, 61)

1080
(571, 246, 1055, 540)

(662, 593, 233, 135)

1080
(774, 186, 912, 488)

(1054, 591, 138, 111)

1080
(454, 395, 40, 16)

(404, 601, 458, 177)

1080
(322, 73, 148, 308)

(233, 131, 87, 14)

1080
(751, 359, 561, 541)

1080
(1490, 289, 223, 487)

(1515, 553, 73, 76)

1080
(11, 413, 1410, 652)

(838, 221, 538, 325)

(1337, 306, 211, 152)

(1398, 365, 43, 31)

(1491, 329, 277, 215)

(1571, 417, 58, 41)

(926, 442, 43, 31)

(1745, 319, 119, 103)

1080
(261, 446, 767, 611)

(373, 801, 139, 123)

1080
(119, 9, 1554, 1056)

1080
(1023, 157, 722, 549)

(1191, 555, 129, 85)

1080
(847, 261, 461, 226)

(917, 448, 62, 26)

(1289, 312, 464, 308)

(1431, 459, 90, 53)

(1727, 340, 124, 121)

(1826, 296, 70, 96)

1080
(993, 325, 690, 432)

(1153, 528, 104, 71)

(658, 393, 210, 254)

(823, 491, 42, 41)

(685

(194, 212, 639, 526)

(509, 576, 142, 64)

1080
(1199, 367, 347, 312)

(1259, 539, 84, 33)

(1232, 256, 258, 193)

(1272, 352, 57, 22)

1080
(1509, 215, 188, 168)

(1556, 297, 66, 27)

(900, 579, 168, 91)

(730, 266, 943, 629)

1080
(2, 381, 1462, 696)

(1398, 329, 197, 137)

(1444, 367, 52, 27)

(1011, 308, 147, 122)

1080
(1396, 97, 436, 498)

1080
(1196, 271, 317, 195)

(1270, 338, 65, 49)

(1306, 389, 186, 232)

(1320, 483, 44, 34)

1080
(196, 485, 335, 236)

(684, 385, 203, 262)

(1220, 253, 477, 406)

(1032, 593, 205, 100)

1080
(260, 447, 247, 264)

(632, 283, 993, 594)

(656, 575, 139, 130)

(1420, 321, 161, 100)

1080
(1540, 169, 197, 226)

(1272, 343, 411, 324)

(1308, 547, 163, 96)

1080
(1330, 341, 395, 282)

(1380, 447, 117, 96)

1080
(378, 199, 233, 98)

(294, 229, 79, 100)

(636, 303, 1221, 720)

1080
(1359, 147, 405, 426)

(321, 446, 569, 471)

(1748, 285, 101, 147)

1080
(1274, 369, 266, 502)

1080
(129, 25, 472, 467)

(311, 360, 90, 48)

1080
(0, 392, 1140, 682)

(154

(477, 208, 1446, 868)

1080
(178, 436, 353, 300)

(927, 393, 301, 367)

(945, 589, 74, 70)

(1553, 205, 247, 312)

(1353, 320, 216, 141)

1080
(1085, 169, 691, 531)

(1285, 390, 115, 52)

1080
(901, 318, 782, 543)

(1026, 529, 114, 78)

1080
(1694, 325, 147, 130)

(1716, 369, 51, 46)

(1408, 319, 189, 128)

(1614, 349, 133, 216)

(1060, 387, 255, 268)

(1082, 503, 67, 66)

(550, 429, 521, 396)

(572, 649, 151, 132)

1080
(1084, 355, 369, 318)

(1138, 525, 91, 84)

1080
(1038, 29, 775, 576)

1080
(282, 183, 261, 162)

(386, 385, 1517, 692)

1080
(280, 235, 525, 422)

(536, 511, 149, 92)

1080
(404, 121, 1533, 940)

1080
(22, 53, 1637, 940)

1080
(6, 125, 1441, 912)

1080
(354, 315, 1505, 758)

1080
(1398, 297, 175, 124)

(1626, 375, 175, 142)

(1670, 407, 55, 34)

(1678, 249, 169, 180)

1080
(1196, 211, 473, 408)

(1236, 397, 143, 90)

1080
(846, 261, 541, 254)

(1374, 325, 187, 132)

1080
(932, 159, 835, 554)

(1066, 447, 199, 100)

1080
(376, 325, 1143, 602)

(1622, 307, 221, 188)

(1

(250, 203, 203, 169)

1080
(1264, 217, 492, 404)

(1370, 389, 116, 67)

1080
(364, 254, 1289, 796)

(548, 641, 149, 130)

(1556, 311, 218, 184)

(1633, 391, 50, 38)

1080
(169, 128, 296, 316)

1080
(301, 80, 1619, 992)

1080
(1141, 313, 617, 474)

(1304, 632, 134, 80)

(758, 449, 353, 325)

(797, 545, 88, 74)

1080
(449, 283, 1195, 767)

(589, 599, 177, 131)

1080
(1295, 169, 515, 386)

(1447, 406, 95, 60)

1080
(1295, 265, 452, 351)

(1417, 478, 93, 56)

(808, 256, 549, 288)

(916, 430, 63, 48)

(686, 377, 214, 263)

(821, 490, 50, 42)

1080
(1355, 43, 446, 548)

(898, 412, 407, 348)

(979, 550, 99, 83)

1080
(245, 68, 1388, 971)

(1244, 755, 220, 134)

1080
(1409, 314, 368, 283)

(1514, 436, 73, 48)

(811, 253, 564, 296)

(1340, 299, 170, 130)

(923, 439, 46, 42)

1080
(446, 248, 467, 653)

1080
(539, 100, 1372, 966)

1080
(335, 113, 1579, 958)

1080
(1373, 220, 373, 324)

(1492, 352, 80, 57)

1080
(1240, 25, 566, 618)

1080
(952, 362, 530, 526)

1080
(41, 349, 1618, 725)

(1634, 316

(99, 42, 375, 410)

1080
(285, 6, 1430, 916)

1080
(1440, 206, 351, 334)

(1548, 312, 65, 44)

1080
(1369, 334, 364, 291)

(1460, 465, 80, 53)

(1689, 342, 141, 121)

(592, 404, 535, 491)

(604, 677, 105, 68)

(816, 224, 564, 322)

1080
(13, 491, 649, 567)

1080
(0, 9, 1571, 1065)

1080
(5, 33, 1221, 1047)

1080
(1087, 213, 620, 434)

(1266, 419, 104, 64)

1080
(267, 68, 1638, 1003)

1080
(14, 476, 1130, 594)

(996, 345, 224, 383)

(1023, 561, 74, 65)

(1552, 394, 151, 268)

(1580, 551, 48, 59)

1080
(214, 212, 345, 305)

(343, 417, 90, 54)

1080
(1498, 321, 303, 214)

(1603, 422, 57, 47)

1080
(983, 390, 336, 333)

(1057, 571, 91, 59)

1080
(11, 289, 1601, 785)

(1536, 334, 273, 192)

(1624, 413, 51, 16)

1080
(919, 396, 438, 463)

(975, 600, 101, 95)

1080
(354, 258, 1397, 806)

(528, 685, 141, 150)

1080
(1052, 404, 397, 459)

(1084, 611, 95, 109)

1080
(179, 207, 472, 419)

(346, 477, 127, 75)

1080
(331, 270, 1281, 778)

(428, 661, 123, 113)

1080
(456, 286, 163, 280)

1080
(1373,

In [5]:
namespath = os.path.join(yolo_anno_dir, 'obj.names')
with copen(namespath, 'w+', "utf8") as fp:    
    for k, v in sorted( classidx.items(), key=lambda x:x[1] ):
        fp.write(k)
        fp.write(os.linesep)

datapath = os.path.join(yolo_anno_dir, 'obj.data')
context = """classes= {}
train  = yolo/annotate/train.txt
valid  = yolo/annotate/test.txt
names = yolo/annotate/obj.names
backup = backup/""".format(len(classidx))
with open(datapath, 'w+') as fp:
    fp.write(context)

all_images = images.all().values_list('id')
train, test = train_test_split(all_images, test_size=0.1)
print len(train), len(test)
with copen(os.path.join(yolo_anno_dir, 'train.txt'), 'w+', "utf8") as fp:
    for x in train:
        fname = "{}.png".format(x[0])
        fp.write(os.path.join(obj_path, fname)+os.linesep)
        
with copen(os.path.join(yolo_anno_dir, 'test.txt'), 'w+', "utf8") as fp:
    for x in test:
        fname = "{}.png".format(x[0])
        fp.write(os.path.join(obj_path, fname)+os.linesep)


1616 180


In [6]:
def create_cfg(args) :
    return """[net]
batch={}
subdivisions={}
width=608
height=608
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky


#######

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[route]
layers=-9

[convolutional]
batch_normalize=1
size=1
stride=1
pad=1
filters=64
activation=leaky

[reorg]
stride=2

[route]
layers=-1,-4

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={}
activation=linear


[region]
anchors =  0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828
bias_match=1
classes={}
coords=4
num=5
softmax=1
jitter=.3
rescore=1

object_scale=5
noobject_scale=1
class_scale=1
coord_scale=1

absolute=1
thresh = .6
random=1""".format(*args)

In [7]:
nclass = len(classidx)
train_cfg = (64, 8, (nclass+5)*5, nclass)
test_cfg = (1, 1, (nclass+5)*5, nclass)

path = os.path.join(yolo_path, 'yolo-obj-train.cfg')
with open(path, 'w+') as fp:
    fp.write(create_cfg(train_cfg))

path = os.path.join(yolo_path, 'yolo-obj-test.cfg')
with open(path, 'w+') as fp:
    fp.write(create_cfg(test_cfg))

In [8]:
# ./darknet detector train yolo/annotate/obj.data cfg/yolo-obj.cfg darknet19_448.conv.23
len(images)

1796